In [2]:
!pip install scikit-optimize
!pip install scikit-learn
!pip install datasets transformers pandas numpy

from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Load the dataset
df = pd.read_csv("FinancES_train_kaggle.csv", encoding="utf-8")
print("Dataset shape:", df.shape)

# Split into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Load BETO model and tokenizer
model_checkpoint = "dccuchile/bert-base-spanish-wwm-cased"
num_labels = len(df["label"].unique())
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns
train_dataset = train_dataset.remove_columns(["id", "text"])
val_dataset = val_dataset.remove_columns(["id", "text"])

# Compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"f1_macro": f1_score(labels, preds, average="macro")}

# Define the search space
space = [
    Real(1e-6, 1e-4, "log-uniform", name="learning_rate"),
    Categorical([8, 16, 32], name="batch_size"),
    Integer(3, 10, name="num_epochs"),
    Real(1e-4, 1e-2, "log-uniform", name="weight_decay"),
    Categorical([1, 2, 4], name="gradient_accumulation_steps")
]

# Objective function for Bayesian optimization
@use_named_args(space)
def objective(**params):
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

    training_args = TrainingArguments(
        output_dir="beto-checkpoints",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=params["learning_rate"],
        per_device_train_batch_size=params["batch_size"],
        num_train_epochs=params["num_epochs"],
        weight_decay=params["weight_decay"],
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        fp16=True,
        gradient_accumulation_steps=params["gradient_accumulation_steps"],
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    metrics = trainer.evaluate()
    return -metrics["eval_f1_macro"]  # Negative because we want to maximize

# Run Bayesian optimization
result = gp_minimize(objective, space, n_calls=10, random_state=42)

# Print best parameters and score
print("Best parameters:")
for name, value in zip(["learning_rate", "batch_size", "num_epochs", "weight_decay", "gradient_accumulation_steps"], result.x):
    print(f"  {name}: {value}")
print(f"Best F1 Macro: {-result.fun}")

# Train final model with best hyperparameters
best_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
best_training_args = TrainingArguments(
    output_dir="beto-checkpoints-best",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=result.x[0],
    per_device_train_batch_size=result.x[1],
    num_train_epochs=result.x[2],
    weight_decay=result.x[3],
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    fp16=True,
    gradient_accumulation_steps=result.x[4],
    report_to="none"
)

best_trainer = Trainer(
    model=best_model,
    args=best_training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

best_trainer.train()

# Final evaluation and prediction generation
final_metrics = best_trainer.evaluate()
print("F1 Macro final (validación):", final_metrics["eval_f1_macro"])

test_df = pd.read_csv("FinancES_test_kaggle.csv", encoding="utf-8")
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.remove_columns(["id", "text"])

predictions = best_trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)

submission = pd.DataFrame({"id": test_df["id"], "label": preds})
submission.to_csv("submission_beto_optimized.csv", index=False)
print("Archivo 'submission_beto_optimized.csv' creado con las predicciones optimizadas.")


Dataset shape: (6359, 3)


Map:   0%|          | 0/5087 [00:00<?, ? examples/s]

Map:   0%|          | 0/1272 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-2-68eaf8907cf4>:77: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-2-68eaf8907cf4>:116: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  best_trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1 Macro
1,0.731600,0.585081,0.703868
2,0.505600,0.585921,0.716128
3,0.432400,0.714417,0.714762
4,0.321300,0.968699,0.732378
5,0.267800,1.048533,0.716248
6,0.195500,1.162513,0.728566
7,0.205700,1.322175,0.729920
8,0.116800,1.403411,0.709603
9,0.094100,1.477062,0.711847
10,0.106700,1.481527,0.712586


F1 Macro final (validación): 0.732377794557594


Map:   0%|          | 0/1621 [00:00<?, ? examples/s]

Archivo 'submission_beto_optimized.csv' creado con las predicciones optimizadas.
